# Pitch Predict

## Data Acquisition

In [146]:
import pybaseballstats as pyb
import polars as pl

For this test we will only be looking at Max Scherzer's data so find his player id and use that to filter the data. 


In [147]:
# pyb.retrosheet.player_lookup(first_name="Max", last_name="Scherzer")

Get all available pitch by pitch data

In [149]:
# df = df.filter(pl.col("pitcher") == 453286)

In [150]:
# print(df.columns)

In [151]:
# df.sink_parquet("scherzer.parquet")

In [217]:
df = pl.read_parquet("scherzer.parquet")

In [218]:
feature_cols = [
    "pitch_type",
    "game_date",
    "release_speed",
    "release_pos_x",
    "release_pos_z",
    "zone",
    "stand",
    "p_throws",
    "balls",
    "strikes",
    "pfx_x",
    "pfx_z",
    "plate_x",
    "plate_z",
    "on_3b",
    "on_2b",
    "on_1b",
    "outs_when_up",
    "inning",
    "inning_topbot",
    "hc_x",
    "hc_y",
    "vx0",
    "vy0",
    "vz0",
    "ax",
    "ay",
    "az",
    "sz_top",
    "sz_bot",
    "effective_speed",
    "release_spin_rate",
    "release_extension",
    "release_pos_y",
    "at_bat_number",
    "pitch_number",
    "bat_score",
    "fld_score",
    "if_fielding_alignment",
    "of_fielding_alignment",
    "spin_axis",
    "n_thruorder_pitcher",
    "n_priorpa_thisgame_player_at_bat",
    "pitcher_days_since_prev_game",
    "api_break_z_with_gravity",
    "api_break_x_arm",
    "api_break_x_batter_in",
]

In [219]:
df = df.select(feature_cols)

In [220]:
df.shape

(31531, 47)

In [221]:
df.describe()

statistic,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,zone,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in
str,str,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,str,f64,f64,f64
"""count""","""31190""","""31531""",31181.0,31176.0,31176.0,31176.0,"""31531""","""31531""",31531.0,31531.0,31176.0,31176.0,31176.0,31176.0,1951.0,4417.0,7952.0,31531.0,31531.0,"""31531""",4864.0,4864.0,31176.0,31176.0,31176.0,31176.0,31176.0,31176.0,31176.0,31176.0,"""27165""","""26577""","""27177""",31176.0,31531.0,31531.0,31531.0,31531.0,"""26955""","""26955""","""23868""",31531.0,31531.0,"""29682""",31176.0,31176.0,31176.0
"""null_count""","""341""","""0""",350.0,355.0,355.0,355.0,"""0""","""0""",0.0,0.0,355.0,355.0,355.0,355.0,29580.0,27114.0,23579.0,0.0,0.0,"""0""",26667.0,26667.0,355.0,355.0,355.0,355.0,355.0,355.0,355.0,355.0,"""4366""","""4954""","""4354""",355.0,0.0,0.0,0.0,0.0,"""4576""","""4576""","""7663""",0.0,0.0,"""1849""",355.0,355.0,355.0
"""mean""",null,null,89.318101,-3.357038,5.389245,8.840454,null,null,0.804732,0.963909,-0.52092,0.693382,0.003706,2.285297,547072.17837,548624.566901,546664.174673,0.989122,3.779582,null,127.324722,121.270446,9.486847,-130.007011,-2.755124,-8.474538,26.662472,-23.095872,3.414538,1.580566,null,null,null,54.422887,27.471029,2.927849,0.933082,1.767974,null,null,null,1.947385,0.914402,null,2.237408,0.52092,-0.122859
"""std""",null,null,5.875498,0.21505,0.246697,4.266978,null,null,0.933989,0.838817,0.757316,0.65843,0.86956,0.882787,85334.652769,83418.363899,85281.102081,0.811651,2.007045,null,41.721228,42.900778,2.719638,8.479718,2.484475,8.523227,4.234665,8.140726,0.20589,0.128306,null,null,null,0.219058,16.433192,1.760428,1.271124,2.161322,null,null,null,0.841248,0.842157,null,1.044563,0.757316,0.910933
"""min""","""CH""","""2014-03-15""",37.9,-4.1,4.45,1.0,"""L""","""R""",0.0,0.0,-2.81,-1.89,-6.5,-1.58,116539.0,114739.0,114739.0,0.0,1.0,"""Bot""",9.99,9.4,-3.119842,-145.637471,-12.320552,-27.676654,3.078948,-50.650566,2.57,0.97,"""34.4""","""1002""","""4""",52.12,1.0,1.0,0.0,0.0,"""Infield shade""","""Standard""","""0""",1.0,0.0,"""10""",0.02,-2.12,-2.42
"""25%""",null,null,85.0,-3.51,5.22,5.0,null,null,0.0,0.0,-1.1,0.2,-0.57,1.72,489232.0,493351.0,474233.0,0.0,2.0,null,100.68,88.67,7.897356,-137.23199,-4.466242,-14.99,23.249251,-29.914,3.28,1.5,null,null,null,54.29,13.0,1.0,0.0,0.0,null,null,null,1.0,0.0,null,1.34,-0.17,-0.96
"""50%""",null,null,91.2,-3.35,5.39,9.0,null,null,1.0,1.0,-0.83,0.87,0.02,2.33,546318.0,543768.0,546318.0,1.0,4.0,null,127.21,121.49,9.690216,-132.818927,-2.877776,-12.1927,26.695039,-21.371,3.41,1.58,null,null,null,54.5,27.0,3.0,0.0,1.0,null,null,null,2.0,1.0,null,1.83,0.83,-0.26
"""75%""",null,null,94.4,-3.2,5.56,13.0,null,null,1.0,2.0,0.17,1.26,0.6,2.89,607054.0,607732.0,607054.0,2.0,5.0,null,155.44,157.24,11.35668,-123.668889,-1.150975,0.06583,29.988419,-15.842864,3.54,1.66,null,null,null,54.5,40.0,4.0,1.0,3.0,null,null,null,3.0,2.0,null,2.98,1.1,0.81
"""max""","""SL""","""2025-03-29""",99.4,-1.91,6.9,14.0,"""R""","""R""",3.0,2.0,2.12,2.64,3.99,6.7,694384.0,807799.0,807799.0,2.0,9.0,"""Top""",246.71,224.11,18.699557,-73.599,7.843897,17.001735,43.224,1.103051,4.5,2.22,"""99""","""993""","""8.4""",56.45,77.0,13.0,7.0,15.0,"""Strategic""","""Strategic""","""99""",4.0,3.0,"""9""",7.8,2.81,2.81


In [222]:
df = df.drop_nulls("pitch_type")

In [223]:
df = df.sort(["game_date", "at_bat_number", "pitch_number"])

In [ ]:
keep_pitch_types = (
    df.select(pl.col("pitch_type").value_counts(normalize=True))
    .unnest("pitch_type")
    .filter(pl.col("proportion") > 0.01)
    .select(pl.col("pitch_type").unique())
    .to_series()
    .to_list()
)
df = df.with_columns(
    pl.when(pl.col("pitch_type").is_in(keep_pitch_types))
    .then(pl.col("pitch_type"))
    .otherwise(pl.lit("other"))
    .alias("pitch_type"),
)

In [226]:
df = df.with_columns(
    pl.col("pitch_type").shift(-1).alias("next_pitch_type (target)"),
)

In [227]:
df = df.drop_nulls(["pitch_type", "next_pitch_type (target)"])

In [228]:
df.select(pl.col(["pitch_type", "next_pitch_type (target)"])).head(10)

pitch_type,next_pitch_type (target)
str,str
"""FF""","""FF"""
"""FF""","""FF"""
"""FF""","""FF"""
"""FF""","""FF"""
"""FF""","""SL"""
"""SL""","""FF"""
"""FF""","""FF"""
"""FF""","""FF"""
"""FF""","""CH"""


## featurization

In [229]:
df.describe()

statistic,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,zone,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,next_pitch_type (target)
str,str,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,f64,f64,f64,f64,f64,str,str,str,f64,f64,str,f64,f64,f64,str
"""count""","""31189""","""31189""",31171.0,31171.0,31171.0,31171.0,"""31189""","""31189""",31189.0,31189.0,31171.0,31171.0,31171.0,31171.0,1917.0,4347.0,7872.0,31189.0,31189.0,"""31189""",4759.0,4759.0,31171.0,31171.0,31171.0,31171.0,31171.0,31171.0,31171.0,31171.0,"""27155""","""26567""","""27167""",31171.0,31189.0,31189.0,31189.0,31189.0,"""26929""","""26929""","""23863""",31189.0,31189.0,"""29633""",31171.0,31171.0,31171.0,"""31189"""
"""null_count""","""0""","""0""",18.0,18.0,18.0,18.0,"""0""","""0""",0.0,0.0,18.0,18.0,18.0,18.0,29272.0,26842.0,23317.0,0.0,0.0,"""0""",26430.0,26430.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,18.0,"""4034""","""4622""","""4022""",18.0,0.0,0.0,0.0,0.0,"""4260""","""4260""","""7326""",0.0,0.0,"""1556""",18.0,18.0,18.0,"""0"""
"""mean""",null,null,89.329813,-3.357032,5.389088,8.840332,null,null,0.809548,0.967745,-0.520915,0.693283,0.004325,2.285,547613.706312,548699.980906,546597.255716,0.988906,3.780339,null,127.259746,121.370712,9.488209,-130.012623,-2.755978,-8.475455,26.664165,-23.095389,3.414518,1.580542,null,null,null,54.422707,27.466062,2.937959,0.931963,1.765943,null,null,null,1.947033,0.915579,null,2.236901,0.520915,-0.122838,null
"""std""",null,null,5.831489,0.215043,0.246292,4.266937,null,null,0.934681,0.83842,0.757298,0.658272,0.867455,0.882223,85391.467308,82973.062719,85372.77858,0.811734,2.008623,null,41.757673,42.875064,2.717139,8.463066,2.483218,8.523251,4.231466,8.140605,0.205894,0.128295,null,null,null,0.218317,16.438522,1.762867,1.270965,2.155785,null,null,null,0.840997,0.842524,null,1.043475,0.757298,0.910918,null
"""min""","""CH""","""2014-04-02""",71.1,-4.1,4.45,1.0,"""L""","""R""",0.0,0.0,-2.81,-1.89,-4.99,-1.58,116539.0,114739.0,114739.0,0.0,1.0,"""Bot""",9.99,9.4,-1.88492,-145.637471,-12.320552,-27.676654,3.078948,-50.650566,2.57,0.97,"""70.6""","""1002""","""4.4""",52.12,1.0,1.0,0.0,0.0,"""Infield shade""","""Standard""","""0""",1.0,0.0,"""10""",0.02,-2.12,-2.42,"""CH"""
"""25%""",null,null,85.0,-3.51,5.22,5.0,null,null,0.0,0.0,-1.1,0.2,-0.57,1.72,489232.0,493351.0,473724.0,0.0,2.0,null,100.67,88.7,7.897695,-137.23199,-4.466309,-14.99,23.25,-29.914,3.28,1.5,null,null,null,54.29,13.0,1.0,0.0,0.0,null,null,null,1.0,0.0,null,1.34,-0.17,-0.96,null
"""50%""",null,null,91.2,-3.35,5.39,9.0,null,null,1.0,1.0,-0.83,0.87,0.02,2.33,546318.0,543776.0,546318.0,1.0,4.0,null,127.21,121.58,9.690306,-132.820339,-2.87933,-12.194,26.695128,-21.369,3.41,1.58,null,null,null,54.5,27.0,3.0,0.0,1.0,null,null,null,2.0,1.0,null,1.83,0.83,-0.26,null
"""75%""",null,null,94.4,-3.2,5.56,13.0,null,null,1.0,2.0,0.17,1.26,0.6,2.89,607054.0,607461.0,607054.0,2.0,5.0,null,155.44,157.3,11.356905,-123.670222,-1.151062,0.066069,29.989,-15.842772,3.54,1.66,null,null,null,54.5,40.0,4.0,1.0,3.0,null,null,null,3.0,2.0,null,2.98,1.1,0.81,null
"""max""","""other""","""2025-03-29""",99.4,-1.91,6.65,14.0,"""R""","""R""",3.0,2.0,2.12,2.64,3.99,5.96,694384.0,807799.0,807799.0,2.0,9.0,"""Top""",246.71,224.11,18.699557,-103.45619,7.843897,17.001735,43.224,1.103051,4.5,2.22,"""99""","""982""","""8.4""",56.11,77.0,13.0,7.0,15.0,"""Strategic""","""Strategic""","""99""",4.0,3.0,"""9""",5.83,2.81,2.81,"""other"""


In [230]:
df = df.with_columns(
    pl.col("stand").cast(pl.Categorical).to_physical().alias("stand"),
    pl.col("p_throws").cast(pl.Categorical).to_physical().alias("p_throws"),
    pl.col("inning_topbot").cast(pl.Categorical).to_physical().alias("inning_topbot"),
    pl.col("effective_speed").cast(pl.Float32).alias("effective_speed"),
    pl.col("release_spin_rate").cast(pl.Float32).alias("release_spin_rate"),
    pl.col("release_extension").cast(pl.Float32).alias("release_extension"),
    pl.col("if_fielding_alignment")
    .cast(pl.Categorical)
    .to_physical()
    .alias("if_fielding_alignment"),
    pl.col("of_fielding_alignment")
    .cast(pl.Categorical)
    .to_physical()
    .alias("of_fielding_alignment"),
    pl.col("spin_axis").cast(pl.Float32).alias("spin_axis"),
    pl.col("pitcher_days_since_prev_game")
    .cast(pl.Float32)
    .alias("pitcher_days_since_prev_game"),
)
df = df.with_columns(
    pl.col("pitch_type").cast(pl.Categorical).to_physical().alias("pitch_type"),
)

In [231]:
df = df.fill_null(-1)

In [232]:
df.describe()

statistic,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,zone,stand,p_throws,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,effective_speed,release_spin_rate,release_extension,release_pos_y,at_bat_number,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,next_pitch_type (target)
str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""count""",31189.0,"""31189""",31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,31189.0,"""31189"""
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0"""
"""mean""",1.091795,null,89.277681,-3.355672,5.385401,8.834653,0.472506,0.0,0.809548,0.967745,-0.521192,0.692306,0.003745,2.283104,33657.578088,76474.781974,137958.584116,0.988906,3.780339,0.519414,18.570622,17.672039,9.482156,-129.938166,-2.754964,-8.471141,26.648199,-23.082637,3.411971,1.579053,77.283592,1967.38208,5.162012,54.390721,27.466062,2.937959,0.931963,1.765943,0.98352,-0.069736,146.267593,1.947033,0.915579,5.754465,2.235033,0.520038,-0.123344,null
"""std""",1.430469,null,6.220379,0.222309,0.290121,4.272247,0.499252,0.0,0.934681,0.83842,0.757167,0.659338,0.86754,0.88549,133219.911307,192546.871901,241281.682126,0.811734,2.008623,0.499631,48.920333,47.082608,2.728009,9.010147,2.482859,8.522683,4.282103,8.155539,0.231536,0.142448,30.651947,898.30719,2.384835,1.348857,16.438522,1.762867,1.270965,2.155785,1.012008,0.445624,97.378212,0.840997,0.842524,3.971658,1.046066,0.75796,0.910899,null
"""min""",0.0,"""2014-04-02""",-1.0,-4.1,-1.0,-1.0,0.0,0.0,0.0,0.0,-2.81,-1.89,-4.99,-1.58,-1.0,-1.0,-1.0,0.0,1.0,0.0,-1.0,-1.0,-1.88492,-145.637471,-12.320552,-27.676654,-1.0,-50.650566,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,0.0,-1.0,-1.0,-2.12,-2.42,"""CH"""
"""25%""",0.0,null,85.0,-3.51,5.22,5.0,0.0,0.0,0.0,0.0,-1.1,0.2,-0.57,1.72,-1.0,-1.0,-1.0,0.0,2.0,0.0,-1.0,-1.0,7.89466,-137.230765,-4.465492,-14.9893,23.244028,-29.91084,3.28,1.5,83.300003,1561.0,5.8,54.29,13.0,1.0,0.0,0.0,1.0,0.0,50.0,1.0,0.0,5.0,1.34,-0.17,-0.96,null
"""50%""",0.0,null,91.2,-3.35,5.39,9.0,0.0,0.0,1.0,1.0,-0.83,0.87,0.02,2.33,-1.0,-1.0,-1.0,1.0,4.0,1.0,-1.0,-1.0,9.689051,-132.805468,-2.875943,-12.19,26.692,-21.359647,3.41,1.58,87.5,2394.0,6.0,54.5,27.0,3.0,0.0,1.0,1.0,0.0,203.0,2.0,1.0,5.0,1.83,0.83,-0.26,null
"""75%""",2.0,null,94.4,-3.2,5.56,13.0,1.0,0.0,1.0,2.0,0.17,1.26,0.6,2.89,-1.0,-1.0,346795.0,2.0,5.0,1.0,-1.0,-1.0,11.355857,-123.665162,-1.149172,0.064485,29.984284,-15.84,3.54,1.66,93.5,2513.0,6.2,54.5,40.0,4.0,1.0,3.0,1.0,0.0,223.0,3.0,2.0,6.0,2.98,1.1,0.81,null
"""max""",5.0,"""2025-03-29""",99.4,-1.0,6.65,14.0,1.0,0.0,3.0,2.0,2.12,2.64,3.99,5.96,694384.0,807799.0,807799.0,2.0,9.0,1.0,246.71,224.11,18.699557,-1.0,7.843897,17.001735,43.224,1.103051,4.5,2.22,99.0,3590.0,8.4,56.11,77.0,13.0,7.0,15.0,3.0,1.0,359.0,4.0,3.0,48.0,5.83,2.81,2.81,"""other"""


## creating sequences

In [233]:
import numpy as np

In [234]:
SEQ_LEN = 10
feature_cols.remove("game_date")
features = df[feature_cols].to_numpy()
labels = df["next_pitch_type (target)"].to_numpy()

In [235]:
X, y = [], []
for i in range(SEQ_LEN, len(df)):
    X.append(features[i - SEQ_LEN : i])
    y.append(labels[i])
X = np.array(X)
y = np.array(y)

In [236]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)
class_names = list(le.classes_)
print(class_names)

[np.str_('CH'), np.str_('CU'), np.str_('FC'), np.str_('FF'), np.str_('SL'), np.str_('other')]


## model

In [238]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

# --- 1) Split your data (no shuffle) ---
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, shuffle=False
)

# --- 2) One‐hot encode your labels ---
num_classes = len(np.unique(y_train))
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes)
y_val_oh = tf.keras.utils.to_categorical(y_val, num_classes)
y_test_oh = tf.keras.utils.to_categorical(y_test, num_classes)

# --- 3) Compute class‐weights ---
#    so that minority classes get proportionally larger loss weights
integer_train_labels = np.argmax(y_train_oh, axis=1)
class_weights_array = compute_class_weight(
    class_weight="balanced", classes=np.arange(num_classes), y=integer_train_labels
)
class_weight_dict = dict(enumerate(class_weights_array))

# --- 4) Build tf.data pipelines (optional but efficient) ---
batch_size = 32
train_ds = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train_oh))
    .shuffle(5000)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
val_ds = (
    tf.data.Dataset.from_tensor_slices((X_val, y_val_oh))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)
test_ds = (
    tf.data.Dataset.from_tensor_slices((X_test, y_test_oh))
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# --- 5) Revised Model: Bidirectional, Stacked LSTM ---
seq_len, num_feats = X_train.shape[1], X_train.shape[2]

model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=(seq_len, num_feats)),
        tf.keras.layers.LSTM(16),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

# --- 6) Compile with gradient clipping ---
opt = tf.keras.optimizers.Adam(learning_rate=1e-4, clipnorm=1.0)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

# model.summary()

# --- 7) Train with early stopping and class‐weights ---
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    class_weight=class_weight_dict,
    callbacks=[es],
)

# --- 8) Final evaluation on test data ---
test_loss, test_acc = model.evaluate(test_ds, verbose=1)
print(f"Test  Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1174 - loss: 1.6722 - val_accuracy: 0.0466 - val_loss: 1.8685
Epoch 2/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1184 - loss: 1.6472 - val_accuracy: 0.0453 - val_loss: 1.8443
Epoch 3/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1256 - loss: 1.6163 - val_accuracy: 0.0984 - val_loss: 1.8091
Epoch 4/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1633 - loss: 1.6267 - val_accuracy: 0.0992 - val_loss: 1.8054
Epoch 5/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2405 - loss: 1.6304 - val_accuracy: 0.2232 - val_loss: 1.7608
Epoch 6/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2637 - loss: 1.6143 - val_accuracy: 0.2230 - val_loss: 1.7682
Epoch 7/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2695 - loss: 1.6344 - val_accuracy: 0.2219 - val_loss: 1.7723
Epoch 8/30
683/683 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2649 - loss: 1.5895 - val_accuracy: 0.

In [239]:
import numpy as np
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
)


# 1) Gather true labels and preds from a tf.data.Dataset
def dataset_to_arrays(ds):
    """Extract (X_batch, y_batch) from a batched tf.data.Dataset of (X, one-hot y)."""
    ys_true = []
    ys_pred = []
    for Xb, yb in ds:
        # model.predict returns (batch, num_classes) probs
        probs = model.predict(Xb, verbose=0)
        ys_true.append(np.argmax(yb.numpy(), axis=1))
        ys_pred.append(np.argmax(probs, axis=1))
    return (
        np.concatenate(ys_true),
        np.concatenate(ys_pred),
        np.concatenate([model.predict(Xb, verbose=0) for Xb, _ in ds]),
    )


# Validation
y_val_true, y_val_pred, y_val_probs = dataset_to_arrays(val_ds)

from sklearn.metrics import classification_report, confusion_matrix

# Suppose your classes are 0,1,2,3,4  (i.e. le.classes_.shape == 5)
labels = list(range(len(class_names)))  # [0,1,2,3,4]

print("=== Validation Classification Report ===")
print(
    classification_report(
        y_val_true, y_val_pred, labels=labels, target_names=class_names
    )
)

cm = confusion_matrix(y_val_true, y_val_pred, labels=labels)
print("Confusion matrix:\n", cm)


# Top-2 accuracy
top2_preds = np.argsort(y_val_probs, axis=1)[:, -2:]
top2_acc = np.mean([y_val_true[i] in top2_preds[i] for i in range(len(y_val_true))])
print(f"Top-2 Accuracy: {top2_acc:.4f}")

# 2) Repeat on Test set
y_test_true, y_test_pred, y_test_probs = dataset_to_arrays(test_ds)

print("\n=== Test Classification Report ===")
print(
    classification_report(
        y_test_true, y_test_pred, labels=labels, target_names=class_names
    )
)
print("Test Accuracy:", accuracy_score(y_test_true, y_test_pred))

cm_test = confusion_matrix(y_test_true, y_test_pred)
print("Test Confusion matrix:\n", cm_test)

top2_test = np.argsort(y_test_probs, axis=1)[:, -2:]
top2_test_acc = np.mean(
    [y_test_true[i] in top2_test[i] for i in range(len(y_test_true))]
)
print(f"Test Top-2 Accuracy: {top2_test_acc:.4f}")


=== Validation Classification Report ===
              precision    recall  f1-score   support

          CH       0.14      0.12      0.13       676
          CU       0.13      0.05      0.07       447
          FC       0.10      0.52      0.17       460
          FF       0.47      0.31      0.37      2154
          SL       0.22      0.05      0.08       940
       other       0.00      0.00      0.00         0

    accuracy                           0.22      4677
   macro avg       0.18      0.17      0.14      4677
weighted avg       0.30      0.22      0.23      4677

Confusion matrix:
 [[  81   24  307  225   33    6]
 [  40   23  240  120   24    0]
 [  51   16  238  136   17    2]
 [ 272   85 1036  658   79   24]
 [ 122   35  451  275   44   13]
 [   0    0    0    0    0    0]]
Top-2 Accuracy: 0.3729


/Users/nicocarbone/Documents/dev/PitchPredict/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nicocarbone/Documents/dev/PitchPredict/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nicocarbone/Documents/dev/PitchPredict/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo


=== Test Classification Report ===
              precision    recall  f1-score   support

          CH       0.12      0.10      0.11       622
          CU       0.09      0.03      0.04       561
          FC       0.10      0.52      0.16       441
          FF       0.46      0.28      0.35      2142
          SL       0.18      0.06      0.09       911
       other       0.00      0.00      0.00         0

    accuracy                           0.20      4677
   macro avg       0.16      0.16      0.12      4677
weighted avg       0.28      0.20      0.21      4677

Test Accuracy: 0.20440453282018387
Test Confusion matrix:
 [[  60   19  309  189   36    9]
 [  59   16  281  153   43    9]
 [  40   21  228  118   26    8]
 [ 238   87 1060  601  123   33]
 [  88   42  459  249   51   22]
 [   0    0    0    0    0    0]]
Test Top-2 Accuracy: 0.3592


/Users/nicocarbone/Documents/dev/PitchPredict/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nicocarbone/Documents/dev/PitchPredict/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/nicocarbone/Documents/dev/PitchPredict/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo